#### DeepStream 8.0 - Multi-Class Object Detection with Tracking & Attribute Classification

This notebook demonstrates a comprehensive DeepStream pipeline for:
- **Primary object detection** (vehicles, persons, bicycles, road signs)
- **Object tracking** across frames
- **Secondary inference** for vehicle attributes (color, make, type)
- **On-Screen Display** with attribute overlay


**Pipeline Flow:**
```
filesrc → h264parse → nvv4l2decoder → nvstreammux → 
pgie (primary detector) → nvtracker → 
sgie1 (car type) → sgie2 (car make)  → 
nvvideoconvert → nvdsosd → nvvideoconvert → capsfilter → 
videoconvert → x264enc → h264parse → mp4mux → filesink
```



## Introduction to Multi-DNN pipeline 

#In the previous notebook we learnt on how to make a simple DeepStream pipeline for object detection. In this notebook we take the idea forward and learn to build a  multi-class object detection,tracking and attribute classification pipeline


**Contents of this Notebook :**



![test2](images/test2.png)

We can explore the architecture diagram of the application. Here, we have 3 additional models that identify car color, make and type respectively. Plugging in additional models is like adding the original classifier, however there are configuration considerations to take care of.A new idea that we will be using in this notebook is the nvtracker plugin.  

### Changes in configuration

Because these secondary classifiers are only intended to execute on objects that we believe are vehicles, we will need to add new configuration parameters to generate this behavior.Two new parameters, `operate-on-gie-id` and `operate-on-class-ids` will let us control this behavior.

The first, `operate-on-gie-id`, lets us configure a classifier to only execute on objects from a
different classifier. In this case, we will configure the secondary classifier to only execute on
objects detected by the primary classifier. The second, `operate-on-class-ids`, lets us configure a
classifier to only execute on objects of a specific class. By combining these two, our secondary
classifiers will be configured to only evaluate the make, model, and color of objects classified as
cars by our primary model.



In [1]:
# Import Required Libraries
import sys
import time

sys.path.append('/opt/nvidia/deepstream/deepstream-8.0/sources/deepstream_python_apps/apps')

import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst, GLib
from common.bus_call import bus_call
import pyds

Gst.init(None)



[]

In [2]:

# Object class IDs
PGIE_CLASS_ID_VEHICLE = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_PERSON = 2
PGIE_CLASS_ID_ROADSIGN = 3

# Input/Output paths
INPUT_VIDEO_NAME = '/opt/nvidia/deepstream/deepstream-8.0/samples/streams/sample_720p.h264'
OUTPUT_VIDEO_NAME = '/app/notebooks/ds_out_multiclass.mp4'

PGIE_CONFIG_FILE = '/app/notebooks/dstest2_pgie_config.txt'

SGIE1_CONFIG_FILE = '/app/notebooks/dstest2_sgie1_config.txt'  # Car make
SGIE2_CONFIG_FILE = '/app/notebooks/dstest2_sgie2_config.txt'  # Car type

TRACKER_LIB = '/opt/nvidia/deepstream/deepstream-8.0/lib/libnvds_nvmultiobjecttracker.so'

print(f" Input video: {INPUT_VIDEO_NAME}")
print(f" Output video: {OUTPUT_VIDEO_NAME}")
print(f"  Primary GIE config: {PGIE_CONFIG_FILE}")
print(f"  SGIE1 config (Color): {SGIE1_CONFIG_FILE}")
print(f"  SGIE2 config (Make/Type): {SGIE2_CONFIG_FILE}")
print(f" Tracker library: {TRACKER_LIB}")


 Input video: /opt/nvidia/deepstream/deepstream-8.0/samples/streams/sample_720p.h264
 Output video: /app/notebooks/ds_out_multiclass.mp4
  Primary GIE config: /app/notebooks/dstest2_pgie_config.txt
  SGIE1 config (Color): /app/notebooks/dstest2_sgie1_config.txt
  SGIE2 config (Make/Type): /app/notebooks/dstest2_sgie2_config.txt
 Tracker library: /opt/nvidia/deepstream/deepstream-8.0/lib/libnvds_nvmultiobjecttracker.so


In [3]:

def make_elm_or_print_err(factoryname, name, printedname, detail=""):
    """Create a GStreamer element or print error message"""
    print(f"Creating {printedname}...")
    elm = Gst.ElementFactory.make(factoryname, name)
    if not elm:
        sys.stderr.write(f"Unable to create {printedname}\n")
    if detail:
            sys.stderr.write(detail)
    return elm

print(" Helper function defined")


 Helper function defined


In [4]:

print("\n" + "="*60)
print("CREATING PIPELINE")
print("="*60)
pipeline = Gst.Pipeline()
if not pipeline:
    sys.stderr.write("Unable to create Pipeline\n")
source = make_elm_or_print_err("filesrc", "file-source", "Source")
h264parser = make_elm_or_print_err("h264parse", "h264-parser", "H264 Parser")
decoder = make_elm_or_print_err("nvv4l2decoder", "nvv4l2-decoder", "NV Decoder")
streammux = make_elm_or_print_err("nvstreammux", "stream-muxer", "Stream Muxer")
pgie = make_elm_or_print_err("nvinfer", "primary-inference", "Primary Inference")
tracker = make_elm_or_print_err("nvtracker", "tracker", "NV Tracker")

sgie1 = make_elm_or_print_err("nvinfer", "secondary1-nvinference-engine", "Secondary Inference 1 (Color)")
sgie2 = make_elm_or_print_err("nvinfer", "secondary2-nvinference-engine", "Secondary Inference 2 (Make/Type)")

nvvidconv = make_elm_or_print_err("nvvideoconvert", "convertor", "NV Video Converter 1")
nvosd = make_elm_or_print_err("nvdsosd", "onscreendisplay", "On-Screen Display")
nvvidconv2 = make_elm_or_print_err("nvvideoconvert", "convertor2", "NV Video Converter 2")
capsfilter = make_elm_or_print_err("capsfilter", "caps", "Caps Filter")
sw_videoconvert = make_elm_or_print_err("videoconvert", "sw-videoconvert", "Software Video Converter")

# H100: Use x264enc software encoder
encoder = make_elm_or_print_err("x264enc", "encoder", "H264 Software Encoder")
h264parser2 = make_elm_or_print_err("h264parse", "h264-parser2", "H264 Parser 2")
mp4mux = make_elm_or_print_err("mp4mux", "mp4mux", "MP4 Muxer")
sink = make_elm_or_print_err("filesink", "filesink", "File Sink")

print("\n All elements created successfully!")



CREATING PIPELINE
Creating Source...
Creating H264 Parser...
Creating NV Decoder...
Creating Stream Muxer...
Creating Primary Inference...
Creating NV Tracker...
Creating Secondary Inference 1 (Color)...
Creating Secondary Inference 2 (Make/Type)...
Creating NV Video Converter 1...
Creating On-Screen Display...
Creating NV Video Converter 2...
Creating Caps Filter...
Creating Software Video Converter...
Creating H264 Software Encoder...
Creating H264 Parser 2...
Creating MP4 Muxer...
Creating File Sink...

 All elements created successfully!


In [5]:
# Configure element properties

print("\n" + "="*60)
print("CONFIGURING ELEMENTS")
print("="*60)

# Source: Set input file
source.set_property('location', INPUT_VIDEO_NAME)
print(f"Source location: {INPUT_VIDEO_NAME}")

# Streammux: Set batch properties
streammux.set_property('width', 1920)
streammux.set_property('height', 1080)
streammux.set_property('batch-size', 1)
streammux.set_property('batched-push-timeout', 4000000)
print("Stream muxer: 1920x1080, batch-size=1")

# Primary inference: Set config file
pgie.set_property('config-file-path', PGIE_CONFIG_FILE)
print(f"Primary GIE config: {PGIE_CONFIG_FILE}")

# Tracker: Configure tracker properties
tracker.set_property('tracker-width', 640)
tracker.set_property('tracker-height', 384)
tracker.set_property('ll-lib-file', TRACKER_LIB)
tracker.set_property('ll-config-file', '/opt/nvidia/deepstream/deepstream-8.0/samples/configs/deepstream-app/config_tracker_NvDCF_perf.yml')
tracker.set_property('gpu-id', 0)
print("Tracker configured: NvDCF")

# Secondary inference 1: Car make
sgie1.set_property('config-file-path', SGIE1_CONFIG_FILE)
print(f"SGIE1 config (Type): {SGIE1_CONFIG_FILE}")

# Secondary inference 2: Car type
sgie2.set_property('config-file-path', SGIE2_CONFIG_FILE)
print(f"SGIE2 config (Make/Type): {SGIE2_CONFIG_FILE}")
caps = Gst.Caps.from_string("video/x-raw, format=I420")
capsfilter.set_property("caps", caps)
print("Caps filter: I420 format")
encoder.set_property('bitrate', 4000)  # x264enc uses kbps
encoder.set_property('speed-preset', 'ultrafast')
encoder.set_property('tune', 'zerolatency')
print("Encoder bitrate: 4 Mbps")

# Sink: Set output file
sink.set_property('location', OUTPUT_VIDEO_NAME)
sink.set_property('sync', False)
print(f"Output file: {OUTPUT_VIDEO_NAME}")



CONFIGURING ELEMENTS
Source location: /opt/nvidia/deepstream/deepstream-8.0/samples/streams/sample_720p.h264
Stream muxer: 1920x1080, batch-size=1
Primary GIE config: /app/notebooks/dstest2_pgie_config.txt
Tracker configured: NvDCF
SGIE1 config (Type): /app/notebooks/dstest2_sgie1_config.txt
SGIE2 config (Make/Type): /app/notebooks/dstest2_sgie2_config.txt
Caps filter: I420 format
Encoder bitrate: 4 Mbps
Output file: /app/notebooks/ds_out_multiclass.mp4


In [6]:
print("\n" + "="*60)
print("BUILDING PIPELINE")
print("="*60)

# Add all elements to pipeline
print("Adding elements to pipeline...")
pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(tracker)
pipeline.add(sgie1)
pipeline.add(sgie2)
pipeline.add(nvvidconv)
pipeline.add(nvosd)
pipeline.add(nvvidconv2)
pipeline.add(capsfilter)
pipeline.add(sw_videoconvert)  # H100: Software converter
pipeline.add(encoder)
pipeline.add(h264parser2)
pipeline.add(mp4mux)
pipeline.add(sink)
print("All elements added")

# Link elements
print("\nLinking elements...")
source.link(h264parser)
h264parser.link(decoder)

# Create pads for streammux (special case)
sinkpad = streammux.get_request_pad("sink_0")
if not sinkpad:
    sys.stderr.write("Unable to get sink pad of streammux\n")
srcpad = decoder.get_static_pad("src")
if not srcpad:
    sys.stderr.write("Unable to get source pad of decoder\n")
srcpad.link(sinkpad)
streammux.link(pgie)
pgie.link(tracker)
tracker.link(sgie1)
sgie1.link(sgie2)
sgie2.link(nvvidconv)
nvvidconv.link(nvosd)
nvosd.link(nvvidconv2)
nvvidconv2.link(capsfilter)
capsfilter.link(sw_videoconvert)  # H100: GPU→CPU
sw_videoconvert.link(encoder)  # H100: x264enc reads CPU memory
encoder.link(h264parser2)
h264parser2.link(mp4mux)
mp4mux.link(sink)

print("All elements linked")
print("\nPipeline built successfully")



BUILDING PIPELINE
Adding elements to pipeline...
All elements added

Linking elements...
All elements linked

Pipeline built successfully


/tmp/ipykernel_267/3754683117.py:32: DeprecationWarning: Gst.Element.get_request_pad is deprecated
  sinkpad = streammux.get_request_pad("sink_0")


In [7]:
# Define metadata probe function with secondary classifier support

def osd_sink_pad_buffer_probe(pad, info, u_data):
    """Callback function to process metadata from each frame including secondary classifiers"""
    
    # Initialize object counter
    obj_counter = {
        PGIE_CLASS_ID_VEHICLE: 0,
        PGIE_CLASS_ID_PERSON: 0,
        PGIE_CLASS_ID_BICYCLE: 0,
        PGIE_CLASS_ID_ROADSIGN: 0
    }
    
    frame_number = 0
    num_rects = 0
    
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer")
        return Gst.PadProbeReturn.OK

    # Retrieve batch metadata from buffer
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    
    while l_frame is not None:
        try:
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        
        frame_number = frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj = frame_meta.obj_meta_list
        
        # Process each object
        while l_obj is not None:
            try:
                obj_meta = pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            
            obj_counter[obj_meta.class_id] += 1
            
            # Extract vehicle attributes from secondary classifiers
            if obj_meta.class_id == PGIE_CLASS_ID_VEHICLE:
                vehicle_color = "Unknown"
                vehicle_make_type = "Unknown"
                
                # Iterate through classifier metadata
                l_classifier = obj_meta.classifier_meta_list
                while l_classifier is not None:
                    try:
                        classifier_meta = pyds.NvDsClassifierMeta.cast(l_classifier.data)
                        l_label = classifier_meta.label_info_list
                        
                        while l_label is not None:
                            try:
                                label_info = pyds.NvDsLabelInfo.cast(l_label.data)
                                
                                if classifier_meta.unique_component_id == 2:  
                                    vehicle_color = label_info.result_label
                                elif classifier_meta.unique_component_id == 3:
                                    vehicle_make_type = label_info.result_label
                                
                                l_label = l_label.next
                            except StopIteration:
                                break
                        
                        l_classifier = l_classifier.next
                    except StopIteration:
                        break
                
                # Add text display for vehicle attributes
                if vehicle_color != "Unknown" or vehicle_make_type != "Unknown":
                    display_meta = pyds.nvds_acquire_display_meta_from_pool(batch_meta)
                    display_meta.num_labels = 1
                    text_params = display_meta.text_params[0]
                    
                    # Create attribute text
                    attr_text = f"ID:{obj_meta.object_id} | {vehicle_color} | {vehicle_make_type}"
                    text_params.display_text = attr_text
                    
                    # Position text near the object bounding box
                    text_params.x_offset = int(obj_meta.rect_params.left)
                    text_params.y_offset = int(obj_meta.rect_params.top) - 30
                    
                    # Text styling
                    text_params.font_params.font_name = "Serif"
                    text_params.font_params.font_size = 10
                    text_params.font_params.font_color.set(1.0, 1.0, 0.0, 1.0)  # Yellow
                    text_params.set_bg_clr = 1
                    text_params.text_bg_clr.set(0.0, 0.0, 0.0, 0.7)  # Semi-transparent black
                    
                    pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
                    
                    # Print vehicle info to console
                    print(f"  Vehicle ID:{obj_meta.object_id} - Color:{vehicle_color}, Make/Type:{vehicle_make_type}")
            
            try:
                l_obj = l_obj.next
            except StopIteration:
                break
        
        # Add frame summary display metadata
        display_meta = pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        
        # Set display text
        py_nvosd_text_params.display_text = "Frame={} Objects={} Vehicles={} Persons={}".format(
            frame_number, num_rects, 
            obj_counter[PGIE_CLASS_ID_VEHICLE], 
            obj_counter[PGIE_CLASS_ID_PERSON]
        )
        
        # Position and style
        py_nvosd_text_params.x_offset = 10
        py_nvosd_text_params.y_offset = 12
        py_nvosd_text_params.font_params.font_name = "Serif"
        py_nvosd_text_params.font_params.font_size = 10
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)  # White
        py_nvosd_text_params.set_bg_clr = 1
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)  # Black background
        
        # Print frame summary to console
        print(pyds.get_string(py_nvosd_text_params.display_text))
        
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        
        try:
            l_frame = l_frame.next
        except StopIteration:
            break
    
    return Gst.PadProbeReturn.OK



In [8]:
# Attach probe to OSD element

osdsinkpad = nvosd.get_static_pad("sink")
if not osdsinkpad:
    sys.stderr.write("Unable to get sink pad of nvosd\n")
else:
    osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)
    print(" Metadata probe attached to OSD element")


 Metadata probe attached to OSD element


In [9]:
# Setup bus message handler

# Create event loop
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

print(" Bus message handler configured")


 Bus message handler configured


In [10]:
# Run the pipeline

print("\n" + "="*60)
print("STARTING PIPELINE")
print("="*60)
print(f"Processing: {INPUT_VIDEO_NAME}")
print("="*60 + "\n")

start_time = time.time()

# Start pipeline
ret = pipeline.set_state(Gst.State.PLAYING)
if ret == Gst.StateChangeReturn.FAILURE:
    print(" ERROR: Unable to set pipeline to PLAYING state")
else:
    try:
        # Run event loop (blocks until EOS or error)
        loop.run()
    except KeyboardInterrupt:
        print("\n Interrupted by user")
    except Exception as e:
        print(f"\n Error: {e}")
    finally:
        # Cleanup
        print("\nCleaning up...")
        pipeline.set_state(Gst.State.NULL)
        
        elapsed_time = time.time() - start_time
        print(f"\n" + "="*60)
        print(f" PIPELINE COMPLETED")
        print(f"  Time elapsed: {elapsed_time:.2f} seconds")
        print(f" Output saved to: {OUTPUT_VIDEO_NAME}")
        print("="*60)



STARTING PIPELINE
Processing: /opt/nvidia/deepstream/deepstream-8.0/samples/streams/sample_720p.h264



0:00:02.713112463   267      0x40a3db0 WARN                 nvinfer gstnvinfer.cpp:682:gst_nvinfer_logger:<secondary2-nvinference-engine> NvDsInferContext[UID 3]: Warning from NvDsInferContextImpl::deserializeEngineAndBackend() <nvdsinfer_context_impl.cpp:2097> [UID = 3]: deserialize engine from file :/opt/nvidia/deepstream/deepstream-8.0/samples/models/Secondary_VehicleTypes/resnet18_vehicletypenet_pruned.onnx_b16_gpu0_fp16.engine failed
0:00:02.713155396   267      0x40a3db0 WARN                 nvinfer gstnvinfer.cpp:682:gst_nvinfer_logger:<secondary2-nvinference-engine> NvDsInferContext[UID 3]: Warning from NvDsInferContextImpl::generateBackendContext() <nvdsinfer_context_impl.cpp:2202> [UID = 3]: deserialize backend context from engine from file :/opt/nvidia/deepstream/deepstream-8.0/samples/models/Secondary_VehicleTypes/resnet18_vehicletypenet_pruned.onnx_b16_gpu0_fp16.engine failed, try rebuild
0:00:02.713162756   267      0x40a3db0 INFO                 nvinfer gstnvinfer.cpp:68

Opening in BLOCKING MODE 
INFO: ../nvdsinfer/nvdsinfer_model_builder.cpp:363 [FullDims Engine Info]: layers num: 2
0   INPUT  kFLOAT input_1:0       3x224x224       min: 1x3x224x224     opt: 16x3x224x224    Max: 16x3x224x224    
1   OUTPUT kFLOAT predictions/Softmax:0 6               min: 0               opt: 0               Max: 0               

INFO: ../nvdsinfer/nvdsinfer_model_builder.cpp:363 [FullDims Engine Info]: layers num: 2
0   INPUT  kFLOAT input_1:0       3x224x224       min: 1x3x224x224     opt: 16x3x224x224    Max: 16x3x224x224    
1   OUTPUT kFLOAT predictions/Softmax:0 20              min: 0               opt: 0               Max: 0               

gstnvtracker: Loading low-level lib at /opt/nvidia/deepstream/deepstream-8.0/lib/libnvds_nvmultiobjecttracker.so
[NvMultiObjectTracker] Initialized


0:02:11.217961026   267      0x40a3db0 WARN                 nvinfer gstnvinfer.cpp:682:gst_nvinfer_logger:<primary-inference> NvDsInferContext[UID 1]: Warning from NvDsInferContextImpl::deserializeEngineAndBackend() <nvdsinfer_context_impl.cpp:2097> [UID = 1]: deserialize engine from file :/opt/nvidia/deepstream/deepstream-8.0/samples/models/Primary_Detector/resnet18_trafficcamnet_pruned.onnx_b1_gpu0_fp16.engine failed
0:02:11.218001338   267      0x40a3db0 WARN                 nvinfer gstnvinfer.cpp:682:gst_nvinfer_logger:<primary-inference> NvDsInferContext[UID 1]: Warning from NvDsInferContextImpl::generateBackendContext() <nvdsinfer_context_impl.cpp:2202> [UID = 1]: deserialize backend context from engine from file :/opt/nvidia/deepstream/deepstream-8.0/samples/models/Primary_Detector/resnet18_trafficcamnet_pruned.onnx_b1_gpu0_fp16.engine failed, try rebuild
0:02:11.218023046   267      0x40a3db0 INFO                 nvinfer gstnvinfer.cpp:685:gst_nvinfer_logger:<primary-inference>

Frame=0 Objects=0 Vehicles=0 Persons=0
Frame=1 Objects=0 Vehicles=0 Persons=0
Frame=2 Objects=13 Vehicles=8 Persons=5
  Vehicle ID:5 - Color:Unknown, Make/Type:largevehicle
  Vehicle ID:10 - Color:mercedes, Make/Type:suv
Frame=3 Objects=14 Vehicles=8 Persons=6
  Vehicle ID:5 - Color:Unknown, Make/Type:largevehicle
  Vehicle ID:10 - Color:mercedes, Make/Type:suv
Frame=4 Objects=12 Vehicles=7 Persons=5
  Vehicle ID:5 - Color:Unknown, Make/Type:largevehicle
  Vehicle ID:10 - Color:mercedes, Make/Type:suv
Frame=5 Objects=12 Vehicles=7 Persons=5
  Vehicle ID:5 - Color:Unknown, Make/Type:largevehicle
  Vehicle ID:10 - Color:mercedes, Make/Type:suv
Frame=6 Objects=12 Vehicles=7 Persons=5
  Vehicle ID:5 - Color:Unknown, Make/Type:largevehicle
  Vehicle ID:10 - Color:mercedes, Make/Type:suv
Frame=7 Objects=11 Vehicles=7 Persons=4
  Vehicle ID:5 - Color:ford, Make/Type:largevehicle
  Vehicle ID:10 - Color:mercedes, Make/Type:suv
Frame=8 Objects=15 Vehicles=10 Persons=5
  Vehicle ID:10 - Color:me

In [11]:
# Check output file

import os

if os.path.exists(OUTPUT_VIDEO_NAME):
    file_size = os.path.getsize(OUTPUT_VIDEO_NAME)
    print(f" Output file exists")
    print(f" Location: {OUTPUT_VIDEO_NAME}")
    print(f" Size: {file_size / (1024*1024):.2f} MB")
    print(f"\n On your host machine: ~/deepstream8/notebooks/ds_out_multiclass.mp4")
else:
    print(f" Output file not found: {OUTPUT_VIDEO_NAME}")


 Output file exists
 Location: /app/notebooks/ds_out_multiclass.mp4
 Size: 22.67 MB

 On your host machine: ~/deepstream8/notebooks/ds_out_multiclass.mp4


In [12]:
# Alternative: Display video with HTML5 player

from IPython.display import HTML
import os

if os.path.exists(OUTPUT_VIDEO_NAME):
    # Create HTML5 video player
    html = f"""
    <div style="text-align: center; margin: 20px;">
        <h3>DeepStream Output Video</h3>
        <video width="800" controls>
            <source src="ds_out_multiclass.mp4" type="video/mp4">
            Your browser does not support the video tag.
        </video>
        <p style="margin-top: 10px;">
            <strong>File:</strong> ds_out_multiclass.mp4 | 
            <strong>Size:</strong> {os.path.getsize(OUTPUT_VIDEO_NAME) / (1024*1024):.2f} MB
        </p>
    </div>
    """
    display(HTML(html))
else:
    print(f"Video not found: {OUTPUT_VIDEO_NAME}")
